<a href="https://colab.research.google.com/github/azhgh22/FaceGeneration/blob/main/models/NADE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Download Data and set up env**

In [1]:
%%capture
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
! unzip challenges-in-representation-learning-facial-expression-recognition-challenge.zip
! rm challenges-in-representation-learning-facial-expression-recognition-challenge.zip
! rm fer2013.tar.gz
! rm icml_face_data.csv
! pip install torchview
! pip install wandb

# **Imports**

In [2]:
import torch # Main PyTorch Library
from torch import nn # Used for creating the layers and loss function
from torch.optim import Adam # Adam Optimizer
import torchvision.transforms as transforms # Transform function used to modify and preprocess all the images
from torch.utils.data import Dataset, DataLoader # Dataset class and DataLoader for creating the objects
from sklearn.preprocessing import LabelEncoder # Label Encoder to encode the classes from strings to numbers
import matplotlib.pyplot as plt # Used for visualizing the images and plotting the training progress
from PIL import Image # Used to read the images from the directory
import pandas as pd # Used to read/create dataframes (csv) and process tabular data
import numpy as np # preprocessing and numerical/mathematical operations
import os # Used to read the images path from the directory
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torchview import draw_graph

device = "cuda" if torch.cuda.is_available() else "cpu" # detect the GPU if any, if not use CPU, change cuda to mps if you have a mac
print("Device available: ", device)

seed = 42

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Device available:  cuda


# **Train Test Split, 48x48 grayscale pictures of different type of emotions**

In [3]:
data = pd.read_csv("train.csv")
data.head()

,emotion,pixels
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [75]:
train, X_test = train_test_split(data, test_size=0.3, random_state=42)
val, test = train_test_split(X_test,test_size=0.5,random_state=42)

pixel_arrays = np.stack(train['pixels'].apply(lambda x: np.fromstring(x, sep=' ')))
train_tensor = torch.tensor(pixel_arrays, dtype=torch.float32).to(device)

pixel_arrays = np.stack(val['pixels'].apply(lambda x: np.fromstring(x, sep=' ')))
val_tensor = torch.tensor(pixel_arrays, dtype=torch.float32).to(device)

pixel_arrays = np.stack(test['pixels'].apply(lambda x: np.fromstring(x, sep=' ')))
test_tensor = torch.tensor(pixel_arrays, dtype=torch.float32).to(device)

# **NADE model for x= {0,1..255}^n**

In [90]:
class NADE256(nn.Module):
  def __init__(self, input_dim=48*48, hidden_dim=512, n_values=256):
    super().__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.n_values = n_values
    self.model = torch.nn.Sequential(
                  torch.nn.Linear(input_dim, hidden_dim),
                  torch.nn.ReLU(),
                  torch.nn.Linear(hidden_dim, n_values),
                  torch.nn.LogSoftmax(dim=-1)
                  ).to(device)

  def forward(self, x):
      """
      x: (batch_size, input_dim), each entry in [0, 255]
      """
      x = x.to(device)
      batch_size, input_dim = x.shape
      losses = 0.0 # torch.tensor([0.0]*input_dim).to(device)

      for i in range(self.input_dim):
        mask = torch.tensor([1]*i + [0]*(self.input_dim-i)).to(device)
        mask = mask.repeat(batch_size,1).to(device)
        masked_input = (mask * x).to(device)

        loss_i = self.model(masked_input)
        # print(loss_i.shape)

        target = x[:, i].long()
        losses += F.nll_loss(loss_i, target, reduction='sum')

      return losses


In [72]:
model = NADE256()
model.forward(train_tensor[:2])[0].backward()

torch.Size([2, 256])
torch.Size([2, 256])
torch.Size([2, 256])
torch.Size([2, 256])
torch.Size([2, 256])
torch.Size([2, 256])


In [91]:
model = NADE256()
optimizer = Adam(model.parameters(), lr=0.001)

EPOCHS = 10
BATCH_SIZE = 64

train_loss = 0.0

for e in range(EPOCHS):
  for i in range(0, len(train_tensor), BATCH_SIZE):
    model.train()
    optimizer.zero_grad()
    batch = train_tensor[i:i+BATCH_SIZE].to(device)
    loss = model.forward(batch)
    train_loss += np.average(loss.detach().cpu().numpy())
    loss.backward()
    optimizer.step()

  print(f"Epoch {e} Train Loss: {train_loss}")



Epoch 0 Train Loss: 690771072.0


KeyboardInterrupt: 

# **Genrating**